In [23]:
name="my_name"
video="video.mp4"
settings="setting.txt"
category="22"
discodiffusion="Disco Diffusion"
private="private"
finished_video_length=10
speed_changed_video_name="speed_changed_output.mp4"
should_create_gist=False
should_upload_video=False
ccmixter_search_url="http://dig.ccmixter.org/search?searchp=instrumental"


### create gist 

In [24]:
'''
HTTP Reuests has following parameters: 
1)Request URL 
2)Header Fields
3)Parameter 
4)Request body
'''
#!/usr/bin/env python
if should_create_gist:

    import requests
    import json

    GITHUB_API="https://api.github.com"
    API_TOKEN=''#this api is in github_api.txt

    #form a request URL
    url=GITHUB_API+"/gists"
    print ("Request URL: %s"%url)

    #print headers,parameters,payload
    headers={'Authorization':'token %s'%API_TOKEN}
    params={'scope':'gist'}

    #with open(settings) as file:
    #   lines = str(file.readlines())
    file=open(settings)
    lines=json.load(file)
    json_formatted_str=json.dumps(lines,indent=4)

    payload={"description":"GIST created by python code","public":True,"files":{name:{"content":json_formatted_str}}}

    #make a requests
    res=requests.post(url,headers=headers,params=params,data=json.dumps(payload))

    #print response --> JSON
    print (res.status_code)
    print (res.url)
    print (res.text)
    j=json.loads(res.text)
    print('j',j)
    # Print created GIST's details
    print ("Gist URL : %s"%(j['url']))
    print ("GIST ID: %s"%(j['id']))
    URL=str("Real Gist URL: https://gist.github.com/hsrmt/%s"%(j['id']))
    print(URL)

### Git Music

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!pip3 install  pyvirtualdisplay selenium webdriver_manager  > /dev/null
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import random

user_input='drums'

if 'http://dig.ccmixter.org' not in user_input:
  user_input="http://dig.ccmixter.org/search?searchp="+user_input

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument("window-size=1400,600")
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=options)

driver.get(user_input)
original_html = driver.page_source
time.sleep(2)
#print("original_html", original_html)
x=str(original_html).split('1 - 40 of ')[1].split('</div><label')
number_of_songs = x[0]
#print("number_of_songs", number_of_songs)
url_with_all_songs="http://dig.ccmixter.org/search?limit="+number_of_songs.replace(',','')+"&searchp=instrumental"
#new_url="http://dig.ccmixter.org/search?limit=10&searchp=instrumental"
#print("url_with_all_songs", url_with_all_songs)

driver.get(url_with_all_songs)
all_songs_html = driver.page_source
time.sleep(2)
y=str(all_songs_html).split('<a class="upload-link song-title" href="/files/')
song_list = []
for i in range(1,len(y)): #use split to get all song names
    song_list.append(y[i].split('</a> <a class="people-link artist-name light-color" ')[0].split('/')[1].split('">')[0])
random_song=random.sample(song_list,1)
print("random_song: ", random_song)

#TODO use the id number before the random_song name to get the xpath of the button that is to be clicked

button_element = driver.find_element_by_xpath("//button[@class='btn btn-warning  btn-lg' and @data-reactid='.0.1.6.0.2.0.0.1.$"+random_song[0]+".2.0']")
WebDriverWait(driver,10).until(EC.element_to_be_clickable(button_element)).click()
driver.switch_to.active_element
time.sleep(2)
html_of_song_popup = driver.page_source
time.sleep(2)
#open text file
text_file = open("data.txt", "w")
 
#write string to file
text_file.write(html_of_song_popup)
 
#close file
text_file.close()
pop_songs=str(html_of_song_popup).split('textarea readonly')[1].split('">')[1].split('</textarea>')[0]
print(pop_songs)
if 'mp3' in html_of_song_popup:
  print('TJ<3Hatice')
#print("html_of_song_popup", html_of_song_popup)
#TODO
#save the credits in html_of_song_popup to a string
#download the mp3 in html_of_song_popup

### Video functions 

In [25]:
def get_video_length(video_name):
    duration = subprocess.check_output(['ffprobe', '-i', video_name, '-show_entries', 'format=duration', '-v', 'quiet', '-of', 'csv=%s' % ("p=0")])
    duration2=str(duration).replace("b'","")
    duration2=str(duration2).replace("\\r\\n'","")
    intduration=round(float(duration2))+1
    return intduration

### Change Speed of a Video

In [26]:
import ffmpeg
import subprocess
#vf = f'select=not(mod(n\,{extract_nth_frame}))'
current_video_length=get_video_length(video)
length_multiplier=str(finished_video_length/current_video_length)
vf="setpts="+length_multiplier+"*PTS"
# subprocess.run(['ffmpeg', '-i', f'{video}', '-vf', f'{vf}', f'{speed_changed_video_name}'], stdout=subprocess.PIPE).stdout.decode('utf-8')
#   #!ffmpeg -i {video_init_path} -vf {vf} -vsync vfr -q:v 2 -loglevel error -stats {videoFramesFolder}/%04d.jpg

cmd = ['ffmpeg', '-i', f'{video}', '-vf', f'{vf}', f'{speed_changed_video_name}']

process = subprocess.Popen(cmd, cwd="./", stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
try:
  if process.returncode != 0:
      print(stderr)
      raise RuntimeError(stderr)

  else:

      print("The video is ready and saved to the images folder")
      #delete all images from label folder

except:
  pass

The video is ready and saved to the images folder


### Add audio to video

In [27]:

import sys
duration = subprocess.check_output(['ffprobe', '-i', speed_changed_video_name, '-show_entries', 'format=duration', '-v', 'quiet', '-of', 'csv=%s' % ("p=0")])
duration2=str(duration).replace("b'","")
duration2=str(duration2).replace("\\r\\n'","")
intduration=round(float(duration2))+1

print(intduration)
input_video = ffmpeg.input(speed_changed_video_name)
input_audio = ffmpeg.input('./song.mp3', t=intduration)

ffmpeg.concat(input_video, input_audio, v=1, a=1).output('./finished_video.mp4').run()


#subprocess.run(ffmpeg -i "video.mp4" -i "song.mp3" -shortest outPutFile.mp4)

#ffmpeg -i "video.mp4" -i "song.mp3" -shortest outPutFile.mp4



11


(None, None)

### Create Youtube Video 

In [28]:
#!/usr/bin/python
if should_upload_video:
  import httplib2
  import os
  import random
  import sys
  import time

  from apiclient.discovery import build
  from apiclient.errors import HttpError
  from apiclient.http import MediaFileUpload
  from oauth2client.client import flow_from_clientsecrets
  from oauth2client.file import Storage
  from oauth2client.tools import argparser, run_flow


  # Explicitly tell the underlying HTTP transport library not to retry, since
  # we are handling retry logic ourselves.
  httplib2.RETRIES = 1

  # Maximum number of times to retry before giving up.
  MAX_RETRIES = 10

  # Always retry when these exceptions are raised.
  RETRIABLE_EXCEPTIONS = (httplib2.HttpLib2Error, IOError)

  # Always retry when an apiclient.errors.HttpError with one of these status
  # codes is raised.
  RETRIABLE_STATUS_CODES = [500, 502, 503, 504]

  # The CLIENT_SECRETS_FILE variable specifies the name of a file that contains
  # the OAuth 2.0 information for this application, including its client_id and
  # client_secret. You can acquire an OAuth 2.0 client ID and client secret from
  # the Google API Console at
  # https://console.developers.google.com/.
  # Please ensure that you have enabled the YouTube Data API for your project.
  # For more information about using OAuth2 to access the YouTube Data API, see:
  #   https://developers.google.com/youtube/v3/guides/authentication
  # For more information about the client_secrets.json file format, see:
  #   https://developers.google.com/api-client-library/python/guide/aaa_client_secrets
  CLIENT_SECRETS_FILE = "client_secrets.json"

  # This OAuth 2.0 access scope allows an application to upload files to the
  # authenticated user's YouTube channel, but doesn't allow other types of access.
  YOUTUBE_UPLOAD_SCOPE = "https://www.googleapis.com/auth/youtube.upload"
  YOUTUBE_API_SERVICE_NAME = "youtube"
  YOUTUBE_API_VERSION = "v3"

  # This variable defines a message to display if the CLIENT_SECRETS_FILE is
  # missing.
  MISSING_CLIENT_SECRETS_MESSAGE = "Error"

  VALID_PRIVACY_STATUSES = ("public", "private", "unlisted")

  class ourargs:
    def __init__(self, **kwargs):
      self.__dict__.update(kwargs)

  def get_authenticated_service(args):
    flow = flow_from_clientsecrets(CLIENT_SECRETS_FILE,
      scope=YOUTUBE_UPLOAD_SCOPE,
      message=MISSING_CLIENT_SECRETS_MESSAGE)

    #storage = Storage("%s-oauth2.json" % sys.argv[0])
    storage = Storage("upload_video.py-oauth2.json")
    credentials = storage.get()

    # if credentials is None or credentials.invalid:
    #   credentials = run_flow(flow, storage, args)

    return build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
      http=credentials.authorize(httplib2.Http()))

  def initialize_upload(youtube, options):
    tags = None
    if options.keywords:
      tags = options.keywords.split(",")

    body=dict(
      snippet=dict(
        title=options.title,
        description=options.description,
        tags=tags,
        categoryId=options.category
      ),
      status=dict(
        privacyStatus=options.privacyStatus
      )
    )

    # Call the API's videos.insert method to create and upload the video.
    insert_request = youtube.videos().insert(
      part=",".join(body.keys()),
      body=body,
      # The chunksize parameter specifies the size of each chunk of data, in
      # bytes, that will be uploaded at a time. Set a higher value for
      # reliable connections as fewer chunks lead to faster uploads. Set a lower
      # value for better recovery on less reliable connections.
      #
      # Setting "chunksize" equal to -1 in the code below means that the entire
      # file will be uploaded in a single HTTP request. (If the upload fails,
      # it will still be retried where it left off.) This is usually a best
      # practice, but if you're using Python older than 2.6 or if you're
      # running on App Engine, you should set the chunksize to something like
      # 1024 * 1024 (1 megabyte).
      media_body=MediaFileUpload(options.file, chunksize=-1, resumable=True)
    )

    resumable_upload(insert_request)

  # This method implements an exponential backoff strategy to resume a
  # failed upload.
  def resumable_upload(insert_request):
    response = None
    error = None
    retry = 0
    while response is None:
      try:
        print ("Uploading file...")
        status, response = insert_request.next_chunk()
        if response is not None:
          if 'id' in response:
            print ("Video id '%s' was successfully uploaded." % response['id'])
          else:
            exit("The upload failed with an unexpected response: %s" % response)
      except HttpError as e:
        if e.resp.status in RETRIABLE_STATUS_CODES:
          error = "A retriable HTTP error %d occurred:\n%s" % (e.resp.status,
                                                              e.content)
        else:
          raise
      except RETRIABLE_EXCEPTIONS as e:
        error = "A retriable error occurred: %s" % e

      if error is not None:
        print (error)
        retry += 1
        if retry > MAX_RETRIES:
          exit("No longer attempting to retry.")

        max_sleep = 2 ** retry
        sleep_seconds = random.random() * max_sleep
        print ("Sleeping %f seconds and then retrying..." % sleep_seconds)
        time.sleep(sleep_seconds)

  if __name__ == '__main__':
    # argparser.add_argument("--file", required=True, help="Video file to upload")
    # argparser.add_argument("--title", help="Video title", default="Test Title")
    # argparser.add_argument("--description", help="Video description",
    #   default="Test Description")
    # argparser.add_argument("--category", default="22",
    #   help="Numeric video category. " +
    #     "See https://developers.google.com/youtube/v3/docs/videoCategories/list")
    # argparser.add_argument("--keywords", help="Video keywords, comma separated",
    #   default="")
    # argparser.add_argument("--privacyStatus", choices=VALID_PRIVACY_STATUSES,
    #   default=VALID_PRIVACY_STATUSES[0], help="Video privacy status.")
    # args = argparser.parse_args()
    args=ourargs(file=video,title=name,description=URL,category=category,keywords=discodiffusion,privacyStatus=private)
    # args.file=video
    # args.title=name
    # args.description=URL
    


    if not os.path.exists(args.file):
      exit("Please specify a valid file using the --file= parameter.")

    youtube = get_authenticated_service(args)
    try:
      initialize_upload(youtube, args)
    except HttpError as e:
      print ("An HTTP error %d occurred:\n%s" % (e.resp.status, e.content))